# AI Community @ Семинар № 3, весна 2018
## Условные языковые модели
Языковая модель называется __условной__, если она зависит от некоторого контекста:
$$ P(w_1, \dots, w_n\ |\ x), x\ —\ \text{контекст}$$
На практике контекстом может быть все что угодно, например — это может текст на другом языке, курс биткоина, звезды на небе и прочее. Воспользовавшись предыдущими выкладками, получаем:
$$ P(w_1, \dots, w_n\ |\ x) = \prod_{t=1}^n P(w_t\ |\ x, w_1, \dots, w_{t-1})$$
В такой модели можно считать $x$ за вход, а $w_1, \dots, w_n$ за текстовый ответ. Ничего не напоминает?

## Рекуррентные нейронные сети
Если нейронная сеть умеет работать с последовательностями (обрабатывает поэлементно), то она называется рекуррентной. Такое название она имеет из-за механизма обработки — информация, полученная при обработке элемента используется для обработки следующего. 

Помимо этого, если на конце нейронной сети разместить softmax, то нейронная сеть как раз станет языковой моделью!

In [1]:
import numpy as np
from random import sample

from util import read_horoscopes
from keras.models import Sequential
from keras.layers import LSTM, GRU, TimeDistributed, Activation, Dense, Dropout

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
HIDDEN_DIM = 100
LAYER_NUM = 3
SEQ_LENGTH = 100
BATCH_SIZE=128
GENERATE_LENGTH=100

START_TOKEN = '<'
END_TOKEN = '>'

In [3]:
data = read_horoscopes(tokenize=True).split('\n')

In [11]:
chars = list(sorted(set(' '.join(data) + START_TOKEN + END_TOKEN)))
ix_to_char = {ix: char for ix, char in enumerate(chars)}
char_to_ix = {char: ix for ix, char in enumerate(chars)}

VOCAB_SIZE = len(chars)

In [12]:
X = np.zeros((len(data), SEQ_LENGTH, VOCAB_SIZE))
y = np.zeros((len(data), SEQ_LENGTH, VOCAB_SIZE))
for i in range(0, len(data)):
    X_sequence = data[i][:SEQ_LENGTH-2]
    X_sequence = X_sequence + (' ' * ((SEQ_LENGTH-2) - len(X_sequence)))
    X_sequence = START_TOKEN + X_sequence + END_TOKEN
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = data[i][1:SEQ_LENGTH-1]
    y_sequence = y_sequence + (' ' * ((SEQ_LENGTH-2) - len(y_sequence)))
    y_sequence = START_TOKEN + y_sequence + END_TOKEN
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

In [13]:
model = Sequential()
model.add(GRU(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(GRU(HIDDEN_DIM, return_sequences=True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, None, 100)         56100     
_________________________________________________________________
gru_5 (GRU)                  (None, None, 100)         60300     
_________________________________________________________________
gru_6 (GRU)                  (None, None, 100)         60300     
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 86)          8686      
_________________________________________________________________
activation_2 (Activation)    (None, None, 86)          0         
Total params: 185,386
Trainable params: 185,386
Non-trainable params: 0
_________________________________________________________________


In [8]:
def generate_text(model, length, start_char=START_TOKEN):
    ix = [char_to_ix[start_char]]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, VOCAB_SIZE))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        next_ix = np.random.choice(np.arange(VOCAB_SIZE), p=model.predict(X[:, :i+1, :])[0][-1])
        ix.append(next_ix)
        y_char.append(ix_to_char[next_ix])
    return ''.join(y_char)

In [15]:
# Load pretrained model
model.load_weights('checkpoint_100_epoch_500.hdf5')

In [17]:
# Training
nb_epoch = 0
while True:
    print('\n\n')
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, epochs=5)
    nb_epoch += 5
    generate_text(model, GENERATE_LENGTH)
    if nb_epoch % 100 == 0:
        model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))




Epoch 1/5
4380/4380 [==============================] - 65s - loss: 5.5997    
Epoch 2/5
4380/4380 [==============================] - 67s - loss: 3.7249    
Epoch 3/5
4380/4380 [==============================] - 58s - loss: 3.2310    
Epoch 4/5
4380/4380 [==============================] - 72s - loss: 2.8704    
Epoch 5/5
4380/4380 [==============================] - 70s - loss: 2.5944    
<<ии гчСсзия . Сселонасо и рсзсловапикяядь сланоидесдте влиисзпртны злив п пзаизарчотизч иойачаозтио


Epoch 1/5
4380/4380 [==============================] - 58s - loss: 2.3889    
Epoch 2/5
4380/4380 [==============================] - 58s - loss: 2.2030    
Epoch 3/5
4380/4380 [==============================] - 62s - loss: 2.0671    
Epoch 4/5
4380/4380 [==============================] - 66s - loss: 1.9508    
Epoch 5/5
4380/4380 [==============================] - 67s - loss: 1.8534    
<< Дю . Среда . День коске рвебдойностаречикентниосторизеньмт прерраюны пропемние сой юдми Эюрати ув


Epoch 1/5
4

KeyboardInterrupt: 

In [16]:
generate_text(model, 200);

<6!цз2>Ть>к>ф!36«6мьО36!П>Т6уЭу«Тъ>Ть>сзь!!уКъ>Ш>цу3у«ъ6>К!6>!КЖ»ьТъ>!>Эу Р<6Тз6ч>Т6нус«у96 ь36 6О>Ш>Э«ус 6чьч>!>«ьну3учъНзз>к>щ6«уЖ3Тъ>Эу Р<з36>Эу3у«Ж3П>К> Асуч>н6 6>Э«зЭ6 ь3П6>Ш>зн6>!у«П36ТзО>з з>н6

Источники: https://chunml.github.io/ChunML.github.io/project/Creating-Text-Generator-Using-Recurrent-Neural-Network/  
https://github.com/oxford-cs-deepnlp-2017/lectures/blob/master/Lecture%207%20-%20Conditional%20Language%20Modeling.pdf